In [ ]:
%load_ext ext.stackql

In [ ]:
## notebook variables
org_id = "12345"

In [ ]:
## imports and object instantiation
import json, time
from pystackql import StackQL
import pandas as pd
from IPython.display import clear_output, display, Markdown, HTML
from ipytree import Tree, Node

stackql = StackQL()

In [ ]:
## functions
def display_cards(cards_data):
    cards_html = ''
    
    for title, value in cards_data:
        card_template = f"""
        <div style="
            border: 1px solid #e3e3e3;
            border-radius: 4px;
            padding: 20px;
            display: inline-block;
            margin: 5px;
            text-align: center;
            width: 150px;
            background-color: #f7f7f7;">
            <h4 style="margin: 5px 0;">{title}</h4>
            <span style="font-size: 30px; font-weight: bold; color: red;">{value}</span>
        </div>
        """
        cards_html += card_template
    
    display(HTML(cards_html))

def get_icon(resType):
    if resType == "project":
        return 'codepen'
    else:
        return resType

def build_tree_node(df, parent_name, parent_node=None):
    children = df[df['parentDisplayName'] == parent_name]
    
    for _, child in children.iterrows():
        child_node = Node(child['displayName'], opened=False, icon=get_icon(child['resType']))
        if parent_node:
            parent_node.add_node(child_node)
        build_tree_node(df, child['displayName'], child_node)

In [ ]:
## discover folders and projects (creates resources_df)
def get_projects_query(entity_id):
    return f"""
    SELECT displayName, name, parent, projectId 
    FROM google.cloudresourcemanager.projects 
    WHERE parent = '{entity_id}';
    """

def get_folders_query(entity_id):
    return f"""
    SELECT displayName, name, parent 
    FROM google.cloudresourcemanager.folders 
    WHERE parent = '{entity_id}';
    """

def print_overwrite(message):
    clear_output(wait=True)
    print(message)

def get_resources_recursive(entity_id, parent_display_name='organization'):
    resources = []

    # Query for projects
    print_overwrite(f"Searching {entity_id} for projects...")
    project_query = get_projects_query(entity_id)
    project_results = json.loads(stackql.execute(project_query))

    if isinstance(project_results, list):
        print_overwrite(f"Found {len(project_results)} projects in {entity_id}")
        for proj in project_results:
            proj["parentDisplayName"] = parent_display_name
            proj["resType"] = "project"
            resources.append(proj)

    # Query for folders
    print_overwrite(f"Searching {entity_id} for folders...")
    folder_query = get_folders_query(entity_id)
    folder_results = json.loads(stackql.execute(folder_query))

    if isinstance(folder_results, list):
        print_overwrite(f"Found {len(folder_results)} folders in {entity_id}")
        for folder in folder_results:
            folder["parentDisplayName"] = parent_display_name
            folder["resType"] = "folder"
            resources.append(folder)

            # Fetch resources under this folder
            if 'name' in folder:
                resources.extend(get_resources_recursive(folder['name'], folder['displayName']))

    return resources

start_time = time.time()

# Start with the root organization to get all resources
all_resources = get_resources_recursive("organizations/%s" % (org_id))

# Convert to dataframe
resources_df = pd.read_json(json.dumps(all_resources))

# Filter rows where the 'error' column is null
resources_df = resources_df[resources_df['error'].isna()]

# Drop the 'error' column
resources_df.drop('error', axis=1, inplace=True)

# Create root node and build the tree
root = Node("organization", opened=False, icon='building')
build_tree_node(resources_df, "organization", root)

# Display the tree
tree = Tree(nodes=[root])

# Print the elapsed time
elapsed_time = (time.time() - start_time) * 1000
print(f"Total elapsed time: {elapsed_time:.2f} ms")

# Count number of folders and projects
num_folders = len(resources_df[resources_df['resType'] == 'folder'])
num_projects = len(resources_df[resources_df['resType'] == 'project'])

cards_data = [("Number of Projects", num_projects), ("Number of Folders", num_folders)]
display_cards(cards_data)
tree